In [1]:
ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [2]:
# Load FC

import pickle
import numpy as np

pncdir = '/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/PNC/'
lowrankdir = '/home/anton/Documents/Tulane/Research/Work/ContrastiveLearning/PNC/Top10/'
pncdemo = pickle.load(open(f'{pncdir}/demographics.pkl', 'rb'))
no_snps_subs = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/fc_subs_no_snps.pkl', 'rb'))

rest = []
nback = []
emoid = []
restmat = []
nbackmat = []
emoidmat = []
race = []
sex = []
age = []
subids = []

a,b = np.triu_indices(264,1)

for sub in pncdemo['age_at_cnb']:
    if sub in no_snps_subs:
        continue
    try:
        ra = pncdemo['Race'][sub]
        ag = pncdemo['age_at_cnb'][sub]
        se = pncdemo['Sex'][sub]
        if ra not in ['AA', 'EA']:
            continue
        ra = ra == 'AA'
        se = se == 'M'
        r = np.load(f'{pncdir}/fc/{sub}_task-rest_fc.npy')
        n = np.load(f'{pncdir}/fc/{sub}_task-nback_fc.npy')
        e = np.load(f'{pncdir}/fc/{sub}_task-emoid_fc.npy')
        r = remap(vec2mat(r))
        n = remap(vec2mat(n))
        e = remap(vec2mat(e))
        race.append(ra)
        sex.append(se)
        age.append(ag)
        rest.append(r[a,b])
        nback.append(n[a,b])
        emoid.append(e[a,b])
        subids.append(sub)
    except:
        pass

rest = np.stack(rest)
nback = np.stack(nback)
emoid = np.stack(emoid)
race = np.array(race).astype('int')
sex = np.array(sex).astype('int')
age = np.array(age)

print([a.shape for a in [rest, nback, emoid, race, sex, age]])

[(1155, 34716), (1155, 34716), (1155, 34716), (1155,), (1155,), (1155,)]


In [3]:
# Load small SNPs

snps = []
no_snps_subs = []

snps_file = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/snps_all_subs_big.pkl', 'rb'))

for sub in subids:
    if sub not in snps_file:
        print(sub)
        # no_snps_subs.append(sub)
    snps.append(snps_file[sub])

snps = np.stack(snps)
snps[np.isnan(snps)] = 0

print(snps.shape)

(1155, 35621)


In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

class SNPVAE(nn.Module):
    def __init__(self, snpd, ld):
        super(SNPVAE, self).__init__()
        self.enc1 = nn.Linear(snpd*3, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, ld).float().cuda()
        self.dec1 = nn.Linear(ld+2, 1000).float().cuda()
        self.dec20 = nn.Linear(1000, snpd).float().cuda()
        self.dec21 = nn.Linear(1000, snpd).float().cuda()
        self.dec22 = nn.Linear(1000, snpd).float().cuda()

    def enc(self, x):
        x = F.relu(self.enc1(x))
        z = self.enc2(x)
        return z

    def gen(self, n):
        return torch.randn(n, self.ld).float().cuda()/(10**0.5)

    def dec(self, z, sex, race):
        z = torch.cat([z, sex.unsqueeze(1), race.unsqueeze(1)], dim=1)
        x = F.relu(self.dec1(z))
        x0 = self.dec20(x)
        x1 = self.dec21(x)
        x2 = self.dec22(x)
        return torch.stack([x0, x1, x2], dim=1)

def latent_loss(z):
    C = z.T@z
    mu = torch.mean(z, dim=0)
    tgt1 = torch.eye(z.shape[-1]).float().cuda()*len(z)/10
    tgt2 = torch.zeros(z.shape[-1]).float().cuda()
    lossC = rmse(C, tgt1)
    lossmu = rmse(mu, tgt2)
    return lossC, lossmu, C, mu

def decor_loss(z, sex, race):
    sex = sex - torch.mean(sex)
    race = race - torch.mean(race)
    sexp = torch.einsum('n,nz->z', sex, z)
    racep = torch.einsum('n,nz->z', race, z)
    tgt = torch.zeros(z.shape[-1]).float().cuda()
    loss_sex = rmse(sexp, tgt)
    loss_race = rmse(racep, tgt)
    return loss_sex + loss_race

ce = nn.CrossEntropyLoss()

# snpvae = SNPVAE(10433, 30)
snpvae = SNPVAE(35621, 50)
optim = torch.optim.Adam(snpvae.parameters(), lr=1e-4, weight_decay=0)

xclass = torch.from_numpy(snps).long().cuda()
x = torch.cat([xclass == 0, xclass == 1, xclass == 2], dim=1).float().cuda()
sex_t = torch.from_numpy(sex).float().cuda()
race_t = torch.from_numpy(race).float().cuda()

nepochs = 2000
pperiod = 50

for e in range(nepochs):
    optim.zero_grad()
    z = snpvae.enc(x)
    logits = snpvae.dec(z, sex_t, race_t)
    loss_ce = ce(logits, xclass)
    loss_C, loss_mu, _, _ = latent_loss(z)
    loss_decor = decor_loss(z, sex_t, race_t)
    (loss_ce+loss_C+loss_mu+loss_decor).backward()
    # No generative loss because all we would predict are age (no SNP info), sex, race, and task
    optim.step()
    if e % pperiod == 0 or e == nepochs-1:
        print(f'{e} {pretty(loss_ce)} {pretty(loss_C)} {pretty(loss_mu)} {pretty(loss_decor)}')

print('Done')

0 1.0998 19.0505 0.1095 13.3482
50 1.0156 16.161 0.0466 0.8577
100 0.9307 16.1631 0.0436 0.8928
150 0.8669 16.1628 0.0436 0.779
200 0.8331 16.1625 0.0436 0.6407
250 0.8157 16.1622 0.0433 0.9707
300 0.806 16.1621 0.0434 0.6283
350 0.7996 16.1618 0.0431 0.7607
400 0.7955 16.1615 0.0432 0.486
450 0.7925 16.1611 0.043 0.7168
500 0.7904 16.1607 0.0431 0.6168
550 0.7888 16.1602 0.043 0.7891
600 0.7876 16.1598 0.043 0.645
650 0.7867 16.1593 0.043 0.6314
700 0.786 16.1587 0.043 0.6118
750 0.7854 16.1582 0.0429 0.6146
800 0.7849 16.1576 0.043 0.5463
850 0.7845 16.1569 0.0429 0.5758
900 0.7842 16.1562 0.043 0.5339
950 0.7838 16.1556 0.0429 0.5209
1000 0.7836 16.1547 0.043 0.4325
1050 0.7833 16.1538 0.0429 0.4805
1100 0.7831 16.1527 0.0428 0.6416


KeyboardInterrupt: 

In [9]:
# Save VAE

torch.save(snpvae.state_dict(), '/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/vae_snps_big_1000_z30_cov2.torch')

print('Done')

Done


In [52]:
# Test PC imputation

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

xreal = x.detach().cpu().numpy()
print(x.shape)

xtr, xt, _, x2t, _, sx, _, rc = train_test_split(xreal, snps, sex_t, race_t, train_size=0.8)

miss = np.random.permutation(snps.shape[-1])[:15000]
pca = PCA(n_components=5).fit(xtr)

xtt = xt+0
xtt[:,miss] = 3
xtt[:,miss+2*snps.shape[-1]] = 3
xtt[:,miss+2*snps.shape[-1]] = 3

xtt_pca = pca.transform(xtt)
xtt_bak = pca.inverse_transform(xtt_pca)

print(xtt_bak)
xtt_bak = xtt_bak.reshape(xtt_bak.shape[0], 3, snps.shape[-1])
xtt_bak = np.argmax(xtt_bak, axis=1)

print('Done')
acc = np.mean(np.sum(xtt_bak[:,miss] == x2t[:,miss], axis=1))
print(acc)

z = snpvae.enc(torch.from_numpy(xtt).float().cuda())
logits = snpvae.dec(z, sx, rc)
logits = torch.argmax(logits, dim=1).detach().cpu().numpy()

acc = np.mean(np.sum(logits[:,miss] == x2t[:,miss], axis=1))
print(acc)

torch.Size([1155, 106863])
[[0.24966532 0.02186641 0.4018616  ... 0.65877056 0.5578846  1.559467  ]
 [0.24098203 0.02238578 0.3557405  ... 0.56057847 0.4881298  1.1416878 ]
 [0.08759117 0.04335025 0.25025797 ... 0.75888884 0.7121742  1.0762694 ]
 ...
 [0.06981213 0.04644316 0.24043156 ... 0.775421   0.7404393  1.0720156 ]
 [0.23630473 0.02395048 0.3939973  ... 0.6830267  0.58673227 1.5794833 ]
 [0.07110409 0.04516009 0.25177523 ... 0.76285326 0.71239287 1.0467556 ]]
Done
9244.09090909091
9388.82683982684
